# LSTM (Long Short Term Memory) - Architecture Explained (Tanglish)

Pazhaya notebook la namma RNN paathom. Adhula irunda 'Sequence Memory' concept super ah irundhalum, 'Vanishing Gradient Problem' nala adhu long sequence ku work aagadhu.

**Example Problem**: Oru periya paragraph padichitu, kadasiya iruka kelvi ku badhil solla sonna, RNN start la padichadha marandhudum.
Idha solve panna dhaan **LSTM** vandhuchu. Idhu 'Important' vishayatha mattum nyabagam vechikum, unwanted ah 'Forget' pannidum.


## 1. LSTM Architecture: The Gates Concept

LSTM vitruka secret weapon: **Gates**.
Idha oru 'Water Tank' maari imagine pannikonga. Valve (Gate) open panna thanni pogum, close panna pogadhu. Adhe concept dhaan inga information flow ku.

### The 3 Main Gates:
1.  **Forget Gate ($f_t$)**: Pazhaya information la edhu theva illaiyo adha trash panna use aagum. (Example: 'He' vandha udane pazhaya 'She' context ah azhikalam).
2.  **Input Gate ($i_t$)**: Puthiya information la edhu mukkiyam nu mudivu pannum.
3.  **Output Gate ($o_t$)**: Current Context vechu edha veliya sollanum nu mudivu pannum.

### Diagram Flow
```mermaid
graph LR
    Input[Xt] --> ForgetGate
    PrevHidden[Ht-1] --> ForgetGate
    ForgetGate --> CellState[Ct]
    Input --> InputGate
    PrevHidden --> InputGate
    InputGate --> CellState
    CellState --> OutputGate
    OutputGate --> HiddenState[Ht]
```

### Math Behind the Gates
Equation paathu bayapada vendam. Concept purinja podhum.

1.  **Forget**: $f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$
2.  **Input**: $i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$
3.  **Update**: $\tilde{C}_t = \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)$
4.  **Cell State**: $C_t = f_t * C_{t-1} + i_t * \tilde{C}_t$
5.  **Output**: $o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)$
6.  **Hidden State**: $h_t = o_t * \tanh(C_t)$

*Note*: $\sigma$ (Sigmoid) is 0 or 1. (0 = Close Gate, 1 = Open Gate).


In [ ]:
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Random seed for reproducibility
torch.manual_seed(42)
print(f"Using device: {device}")

## 2. PyTorch LSTM Implementation

`nn.LSTM` use panni model create pannuvom. Difference between RNN and LSTM code is simple - Input arguments same dhaan, output la mattum `(HiddenState, CellState)` rendume varum (RNN la HiddenState mattum varum).


In [ ]:
# Hyperparameters
input_size = 5 # E.g., Normalized Word Vector size
hidden_size = 10 # Memory size
num_layers = 2 # Deep LSTM (Stacked)
batch_size = 3 
seq_len = 4 # 'I love Machine Learning' (4 words)

# Create LSTM Model
# Batch First = True na Input shape (Batch, Seq, Features) irukanum
lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

# Inputs
inputs = torch.randn(batch_size, seq_len, input_size)

# Initial States (Hidden State h0, Cell State c0)
h0 = torch.zeros(num_layers, batch_size, hidden_size)
c0 = torch.zeros(num_layers, batch_size, hidden_size)

# Forward Pass
# Output shape: (Batch, Seq, HiddenSize)
# Final States shape: (Num_layers, Batch, HiddenSize)
out, (hn, cn) = lstm(inputs, (h0, c0))

print("Input Data Shape:", inputs.shape)
print("Output Data Shape:", out.shape)
print("Final Hidden State Shape:", hn.shape)
print("Final Cell State Shape:", cn.shape)

## 3. Practical Use Case: Sentiment Analysis (Logic)

LSTM vechu 'Positive' or 'Negative' nu kandupudika use aagum.

**Flow:**
1. Text -> Embedding (Numbers)
2. Embedding -> LSTM Layer
3. LSTM Last Output -> Fully Connected Layer (Classifier)
4. Output -> Sigmoid (Probability 0-1)

Indha architecture dhaan 'Siri', 'Google Translate' start aanapo base ah irundhuchu. But ippo **Transformers** dhaan ruling. Adha next notebook la paapom!

# 📚 LSTM Complete Summary 

**Instructions:** Copy this entire content and paste it as new cells in your LSTM_Architecture_Tanglish.ipynb notebook



## 1️⃣ LSTM Enna? Yean Venum? (What is LSTM? Why Do We Need It?)

### RNN oda Problems:

❌ **Vanishing Gradient Problem** - Long sequences la pazhaya info ah remember pana mudiyathu  
❌ **No Control** - Edha remember pananum, edha forget pananum nu control illa  
❌ **Short-term Memory** - Arambathula padichadha marandhudum

### LSTM - The Solution! 💡

- **LSTM = Long Short-Term Memory**
- RNN oda advanced version
- **Gates** mechanism use panni, edha remember pananum, edha forget pananum nu decide pannum
- Long-term dependencies ah handle pana best!

**Real-Life Example:**

```
"I grew up in France... (100 words later)... I speak fluent ____"
```

- RNN: "France" ah marandhidum → Thappu answer
- LSTM: "France" ah remember vechukum → "French" nu correct ah predict pannum

---

## 2️⃣ LSTM Architecture - The 3 Gates System 🚪

LSTM la **4 main components** irukum:

1. **Cell State (C_t)** - Long-term memory storage (Main highway)
2. **Hidden State (h_t)** - Short-term memory (Current output)
3. **Three Gates** - Information flow control pannum

### Gate 1: Forget Gate (f_t) 🗑️

**Purpose:** Pazhaya information la edha delete pananum nu decide pannum

**Formula:**
$$f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$$

**Example:**

- Input: "The cat was black. **The dog** was white."
- Forget Gate: "cat" pathi info ah forget pannidum (because new subject "dog" vandhuduchu)

**How it works:**

- Sigmoid function use pannum (output: 0 to 1)
- 0 = Completely forget
- 1 = Completely remember

---

### Gate 2: Input Gate (i_t) 📥

**Purpose:** Puthiya information la edha add pananum nu decide pannum

**Formulas:**
$$i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$$
$$\tilde{C}_t = \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)$$

**Example:**

- New word: "beautiful"
- Input Gate: Idhukku important nu mudivu panni cell state ku add pannum

**Two Steps:**

1. **Decide** what to add (sigmoid function)
2. **Create** candidate values (tanh function: -1 to +1)

---

### Gate 3: Output Gate (o_t) 📤

**Purpose:** Current step la enna output kudukanum nu decide pannum

**Formulas:**
$$o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)$$
$$h_t = o_t \cdot \tanh(C_t)$$

**Example:**

- Sentence half complete: "The weather is"
- Output Gate: Current context based ah "sunny" or "rainy" nu predict pannum

---

### Cell State Update 🔄

**Formula:**
$$C_t = f_t \cdot C_{t-1} + i_t \cdot \tilde{C}_t$$

**Breakdown:**

- `f_t · C_{t-1}` → Pazhaya memory la edha keep pananum
- `i_t · C̃_t` → Puthiya info la edha add pananum
- Both ah combine panni new cell state create pannum

---

## 3️⃣ LSTM vs RNN - Quick Comparison

| **Feature**            | **RNN**         | **LSTM**                        |
| ---------------------- | --------------- | ------------------------------- |
| **Memory**             | Short-term only | Long-term + Short-term          |
| **Gates**              | No gates        | 3 Gates (Forget, Input, Output) |
| **Vanishing Gradient** | Problem irukum  | Almost solved                   |
| **Complexity**         | Simple          | Complex (more parameters)       |
| **Training Time**      | Fast            | Slower                          |
| **Use Cases**          | Short sequences | Long sequences                  |



## 4️⃣ LSTM Variants (Different Types)

### a) Vanilla LSTM (Standard LSTM)

- 3 gates + cell state
- Most commonly used version

### b) Peephole LSTM

- Gates ku direct ah cell state access kudukurom
- More control over information flow

### c) Deep LSTM (Stacked LSTM)

- Multiple LSTM layers one after another
- Complex patterns learn pana use aagum

### d) Bidirectional LSTM

- Forward + Backward directions la process pannum
- Full context (past + future) therijrum
- **Use case:** Named Entity Recognition, Speech Recognition

### e) GRU (Gated Recurrent Unit)

- Simplified LSTM version
- Only 2 gates (Update gate + Reset gate)
- Faster training, LSTM ku similar performance



## 5️⃣ GRU vs LSTM - Detailed Comparison

| **Aspect**      | **LSTM**                             | **GRU**                                   |
| --------------- | ------------------------------------ | ----------------------------------------- |
| **Gates**       | 3 Gates                              | 2 Gates                                   |
| **Cell State**  | Separate cell state irukum           | No separate cell state                    |
| **Parameters**  | More parameters                      | Less parameters                           |
| **Training**    | Slower                               | Faster                                    |
| **Memory**      | More memory needed                   | Less memory                               |
| **Performance** | Slightly better for complex tasks    | Similar, sometimes better                 |
| **Best For**    | Long sequences with complex patterns | Simpler sequences, faster training needed |

**When to use what?**

- **LSTM:** Long sequences, complex patterns, more data available
- **GRU:** Faster training needed, less data, simpler patterns

---

## 6️⃣ LSTM Applications - Real World Examples

### 📝 Natural Language Processing (NLP)

- **Language Translation:** English → Tamil conversion
- **Text Generation:** Next word/sentence prediction
- **Sentiment Analysis:** Movie reviews ah positive/negative nu classify
- **Chatbots:** Conversational AI

### 🎤 Speech Recognition

- **Voice Assistants:** Siri, Alexa, Google Assistant
- **Speech-to-Text:** Audio ah text ah convert pannum
- **Voice Commands:** "Hey Google, turn on lights"

### 🖼️ Image Processing

- **Image Captioning:** Image paathu description generate pannum
  - Example: "A dog playing in the park"
- **Video Analysis:** Frame-by-frame sequence analysis

### 🎵 Music Generation

- Notes sequence based ah new music create pannum
- Classical music patterns learn panni generate pannum

### 📈 Time Series Prediction

- **Stock Market:** Price predictions
- **Weather Forecast:** Temperature, rainfall predictions
- **Sales Forecasting:** Future sales predict pannum

### ✍️ Handwriting Recognition

- Handwritten text ah digital text ah convert pannum
- Signature verification



## 7️⃣ LSTM Advantages ✅

🎯 **Long-term Memory** - Vanishing gradient problem ah solve pannum  
🎯 **Selective Memory** - Gates use panni important info mattum remember pannum  
🎯 **Flexible** - Variable length sequences handle panlam  
🎯 **Versatile** - Text, audio, video, time-series data ellathukum use panlam  
🎯 **Better Context** - Past information ah effectively use pannum



## 8️⃣ LSTM Limitations ❌

❌ **Training Time** - RNN vida slow ah train aagum  
❌ **Computational Cost** - More parameters → More memory + processing power venum  
❌ **Still Gradient Issues** - Completely vanishing gradient solve aagala  
❌ **Overfitting** - Small datasets la overfit aaga chances irukum  
❌ **Sequential Processing** - Parallel processing pana mudiyathu (Transformers la idhukku solution)  
❌ **Hardware Demanding** - High memory bandwidth venum



## 9️⃣ Key Formulas - Quick Reference

### All Gates Together:


1. Forget Gate:     f_t = σ(W_f · [h_{t-1}, x_t] + b_f)
2. Input Gate:      i_t = σ(W_i · [h_{t-1}, x_t] + b_i)
3. Candidate:       C̃_t = tanh(W_C · [h_{t-1}, x_t] + b_C)
4. Cell State:      C_t = f_t · C_{t-1} + i_t · C̃_t
5. Output Gate:     o_t = σ(W_o · [h_{t-1}, x_t] + b_o)
6. Hidden State:    h_t = o_t · tanh(C_t)


### Activation Functions:

- **Sigmoid (σ):** 0 to 1 → Gate control (0=close, 1=open)
- **Tanh:** -1 to +1 → Value creation/transformation



## 🔟 Key Takeaways - Summary Points

🎯 **LSTM = RNN + Gates** (Long-term memory solution)  
🎯 **3 Gates:** Forget, Input, Output  
🎯 **2 States:** Cell State (long-term) + Hidden State (short-term)  
🎯 **Solves:** Vanishing gradient problem  
🎯 **Trade-off:** Better performance but slower training  
🎯 **Alternatives:** GRU (faster), Transformers (modern)  
🎯 **Best for:** Sequential data with long-term dependencies



# 💻 PyTorch Implementation Examples

## Example 1: Basic LSTM Layer

import torch
import torch.nn as nn

# LSTM Parameters
input_size = 10      # Input features (e.g., word embedding dimension)
hidden_size = 20     # LSTM cell oda memory capacity
num_layers = 2       # Stacked LSTM layers
batch_size = 5       # Number of sequences
seq_length = 7       # Sequence length (e.g., 7 words)

# Create LSTM Layer
lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

# Random Input Data (Batch, Sequence, Features)
inputs = torch.randn(batch_size, seq_length, input_size)

# Initial States (Hidden State + Cell State)
h0 = torch.zeros(num_layers, batch_size, hidden_size)
c0 = torch.zeros(num_layers, batch_size, hidden_size)

# Forward Pass
output, (hn, cn) = lstm(inputs, (h0, c0))

print("Input Shape:", inputs.shape)          # [5, 7, 10]
print("Output Shape:", output.shape)         # [5, 7, 20]
print("Final Hidden State:", hn.shape)       # [2, 5, 20]
print("Final Cell State:", cn.shape)         # [2, 5, 20]

"""
Explanation:
- output: Each time step oda hidden state (ovvoru word kum)
- hn: Last time step oda hidden state mattum
- cn: Last time step oda cell state (long-term memory)
"""


## Example 2: Sentiment Analysis Model (Complete)


import torch
import torch.nn as nn
import torch.optim as optim

class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout=0.5):
        super(SentimentLSTM, self).__init__()

        # Embedding Layer (Words ah vectors ah convert pannum)
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # LSTM Layer
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           batch_first=True,
                           dropout=dropout if n_layers > 1 else 0)

        # Fully Connected Layer (Classification)
        self.fc = nn.Linear(hidden_dim, output_dim)

        # Dropout for regularization
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        # text shape: [batch_size, seq_length]

        # Embedding
        embedded = self.dropout(self.embedding(text))
        # embedded shape: [batch_size, seq_length, embedding_dim]

        # LSTM
        output, (hidden, cell) = self.lstm(embedded)
        # output shape: [batch_size, seq_length, hidden_dim]
        # hidden shape: [n_layers, batch_size, hidden_dim]

        # Last layer oda last hidden state ah edukurom
        hidden = self.dropout(hidden[-1])  # [batch_size, hidden_dim]

        # Classification
        out = self.fc(hidden)  # [batch_size, output_dim]

        return out

# Model Parameters
VOCAB_SIZE = 10000      # Total unique words
EMBEDDING_DIM = 100     # Word vector size
HIDDEN_DIM = 256        # LSTM memory size
OUTPUT_DIM = 2          # Binary classification (Positive/Negative)
N_LAYERS = 2            # Stacked LSTM
DROPOUT = 0.5

# Create Model
model = SentimentLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, DROPOUT)

print(model)
print(f"\nTotal Parameters: {sum(p.numel() for p in model.parameters()):,}")

"""
Model Architecture:
1. Embedding: Words → Vectors
2. LSTM: Sequential processing
3. Dropout: Prevent overfitting
4. FC: Final classification

Example Usage:
- Input: "This movie is amazing!" → [45, 234, 12, 789, 23]
- Embedding: Each word ID → 100-dim vector
- LSTM: Process sequence → 256-dim hidden state
- FC: 256 → 2 (Positive/Negative scores)
- Softmax: Probabilities
"""

## Example 3: Text Generation (Character-Level LSTM)


import torch
import torch.nn as nn
import numpy as np

class CharLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers):
        super(CharLSTM, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Character Embedding
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # LSTM
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, batch_first=True)

        # Output Layer
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden):
        # x shape: [batch_size, seq_length]

        # Embedding
        embed = self.embedding(x)
        # embed shape: [batch_size, seq_length, embedding_dim]

        # LSTM
        out, hidden = self.lstm(embed, hidden)
        # out shape: [batch_size, seq_length, hidden_dim]

        # Reshape for FC layer
        out = out.reshape(-1, self.hidden_dim)

        # FC
        out = self.fc(out)
        # out shape: [batch_size * seq_length, vocab_size]

        return out, hidden

    def init_hidden(self, batch_size, device):
        # Initialize hidden and cell states
        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        return (h0, c0)

# Example: Text Generation Function
def generate_text(model, start_text, char_to_idx, idx_to_char, length=100, temperature=1.0):
    model.eval()
    device = next(model.parameters()).device

    # Convert start text to indices
    chars = [char_to_idx[c] for c in start_text]
    input_seq = torch.tensor(chars).unsqueeze(0).to(device)  # [1, len(start_text)]

    # Initialize hidden state
    hidden = model.init_hidden(1, device)

    generated = start_text

    with torch.no_grad():
        for _ in range(length):
            # Forward pass
            out, hidden = model(input_seq, hidden)

            # Get last prediction
            logits = out[-1] / temperature
            probs = torch.softmax(logits, dim=0)

            # Sample next character
            next_idx = torch.multinomial(probs, 1).item()
            next_char = idx_to_char[next_idx]

            generated += next_char

            # Update input
            input_seq = torch.tensor([[next_idx]]).to(device)

    return generated

"""
Example Usage:
- Input: "Hello"
- Model predicts next character based on previous characters
- Output: "Hello world, this is an LSTM generated text..."

Temperature parameter:
- Low (0.5): More predictable, conservative
- High (1.5): More random, creative
"""
```

---

## Example 4: Bidirectional LSTM


import torch
import torch.nn as nn

class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(BiLSTM, self).__init__()

        # Bidirectional LSTM
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers,
            batch_first=True,
            bidirectional=True  # Key parameter!
        )

        # FC layer (hidden_size * 2 because bidirectional)
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        # x shape: [batch_size, seq_length, input_size]

        # LSTM
        lstm_out, (hidden, cell) = self.lstm(x)
        # lstm_out shape: [batch_size, seq_length, hidden_size * 2]

        # Use last time step's output
        # lstm_out[:, -1, :] takes the last time step for all batches
        out = self.fc(lstm_out[:, -1, :])

        return out

# Create Bidirectional LSTM
bi_lstm = BiLSTM(input_size=50, hidden_size=128, output_size=3, num_layers=2)

# Test
test_input = torch.randn(32, 10, 50)  # [batch=32, seq=10, features=50]
output = bi_lstm(test_input)
print("Output shape:", output.shape)  # [32, 3]

"""
Bidirectional LSTM Advantages:
- Forward direction: Past context
- Backward direction: Future context
- Both combine → Better understanding

Example:
Sentence: "The cat sat on the mat"
- Forward LSTM: "The" → "cat" → "sat" (past context)
- Backward LSTM: "mat" → "the" → "on" (future context)
- Combined: Full sentence context for each word
"""
```

---

## Example 5: Many-to-Many LSTM (Sequence to Sequence)


import torch
import torch.nn as nn

class Seq2SeqLSTM(nn.Module):
    def __init__(self, input_vocab_size, output_vocab_size, embedding_dim, hidden_dim):
        super(Seq2SeqLSTM, self).__init__()

        # Encoder
        self.encoder_embedding = nn.Embedding(input_vocab_size, embedding_dim)
        self.encoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Decoder
        self.decoder_embedding = nn.Embedding(output_vocab_size, embedding_dim)
        self.decoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Output
        self.fc = nn.Linear(hidden_dim, output_vocab_size)

    def forward(self, src, tgt):
        # src: Source sequence (e.g., English)
        # tgt: Target sequence (e.g., Tamil)

        # Encode
        src_embed = self.encoder_embedding(src)
        _, (hidden, cell) = self.encoder(src_embed)

        # Decode
        tgt_embed = self.decoder_embedding(tgt)
        decoder_out, _ = self.decoder(tgt_embed, (hidden, cell))

        # Output
        output = self.fc(decoder_out)

        return output

"""
Seq2Seq Application: Language Translation
Input: "I love programming" (English)
Process:
1. Encoder: English sentence ah encode pannum → context vector
2. Decoder: Context vector use panni Tamil la decode pannum
Output: "Naan programming ah love pannuren" (Tamil)
"""
```

---

## 🎓 Practice Exercise


# Exercise: Create a simple LSTM for sequence classification

# TODO 1: Define LSTM model with following specs:
#   - Vocab size: 5000
#   - Embedding dimension: 100
#   - Hidden dimension: 128
#   - Output classes: 3 (Multi-class classification)
#   - Layers: 2

# TODO 2: Create sample input data
#   - Batch size: 16
#   - Sequence length: 20

# TODO 3: Forward pass and print shapes

# TODO 4: Calculate total parameters in model

# Solution below (try first before looking!)
```

<details>
<summary>Solution (Click to expand)</summary>

```python
import torch
import torch.nn as nn

# Solution
class SimpleLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, n_layers):
        super(SimpleLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embed = self.embedding(x)
        _, (hidden, _) = self.lstm(embed)
        out = self.fc(hidden[-1])
        return out

# Create model
model = SimpleLSTM(5000, 100, 128, 3, 2)

# Sample input
batch_size, seq_len = 16, 20
x = torch.randint(0, 5000, (batch_size, seq_len))

# Forward pass
output = model(x)
print("Output shape:", output.shape)  # [16, 3]

# Total parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")




# 📚 LSTM Complete Summary 



---

## 1️⃣ LSTM Enna? Yean Venum? (What is LSTM? Why Do We Need It?)

### RNN oda Problems:

❌ **Vanishing Gradient Problem** - Long sequences la pazhaya info ah remember pana mudiyathu  
❌ **No Control** - Edha remember pananum, edha forget pananum nu control illa  
❌ **Short-term Memory** - Arambathula padichadha marandhudum

### LSTM - The Solution! 💡

- **LSTM = Long Short-Term Memory**
- RNN oda advanced version
- **Gates** mechanism use panni, edha remember pananum, edha forget pananum nu decide pannum
- Long-term dependencies ah handle pana best!

**Real-Life Example:**

```
"I grew up in France... (100 words later)... I speak fluent ____"
```

- RNN: "France" ah marandhidum → Thappu answer
- LSTM: "France" ah remember vechukum → "French" nu correct ah predict pannum

---

## 2️⃣ LSTM Architecture - The 3 Gates System 🚪

LSTM la **4 main components** irukum:

1. **Cell State (C_t)** - Long-term memory storage (Main highway)
2. **Hidden State (h_t)** - Short-term memory (Current output)
3. **Three Gates** - Information flow control pannum

### Gate 1: Forget Gate (f_t) 🗑️

**Purpose:** Pazhaya information la edha delete pananum nu decide pannum

**Formula:**
$$f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$$

**Example:**

- Input: "The cat was black. **The dog** was white."
- Forget Gate: "cat" pathi info ah forget pannidum (because new subject "dog" vandhuduchu)

**How it works:**

- Sigmoid function use pannum (output: 0 to 1)
- 0 = Completely forget
- 1 = Completely remember

---

### Gate 2: Input Gate (i_t) 📥

**Purpose:** Puthiya information la edha add pananum nu decide pannum

**Formulas:**
$$i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$$
$$\tilde{C}_t = \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)$$

**Example:**

- New word: "beautiful"
- Input Gate: Idhukku important nu mudivu panni cell state ku add pannum

**Two Steps:**

1. **Decide** what to add (sigmoid function)
2. **Create** candidate values (tanh function: -1 to +1)

---

### Gate 3: Output Gate (o_t) 📤

**Purpose:** Current step la enna output kudukanum nu decide pannum

**Formulas:**
$$o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)$$
$$h_t = o_t \cdot \tanh(C_t)$$

**Example:**

- Sentence half complete: "The weather is"
- Output Gate: Current context based ah "sunny" or "rainy" nu predict pannum

---

### Cell State Update 🔄

**Formula:**
$$C_t = f_t \cdot C_{t-1} + i_t \cdot \tilde{C}_t$$

**Breakdown:**

- `f_t · C_{t-1}` → Pazhaya memory la edha keep pananum
- `i_t · C̃_t` → Puthiya info la edha add pananum
- Both ah combine panni new cell state create pannum

---

## 3️⃣ LSTM vs RNN - Quick Comparison

| **Feature**            | **RNN**         | **LSTM**                        |
| ---------------------- | --------------- | ------------------------------- |
| **Memory**             | Short-term only | Long-term + Short-term          |
| **Gates**              | No gates        | 3 Gates (Forget, Input, Output) |
| **Vanishing Gradient** | Problem irukum  | Almost solved                   |
| **Complexity**         | Simple          | Complex (more parameters)       |
| **Training Time**      | Fast            | Slower                          |
| **Use Cases**          | Short sequences | Long sequences                  |

---

## 4️⃣ LSTM Variants (Different Types)

### a) Vanilla LSTM (Standard LSTM)

- 3 gates + cell state
- Most commonly used version

### b) Peephole LSTM

- Gates ku direct ah cell state access kudukurom
- More control over information flow

### c) Deep LSTM (Stacked LSTM)

- Multiple LSTM layers one after another
- Complex patterns learn pana use aagum

### d) Bidirectional LSTM

- Forward + Backward directions la process pannum
- Full context (past + future) therijrum
- **Use case:** Named Entity Recognition, Speech Recognition

### e) GRU (Gated Recurrent Unit)

- Simplified LSTM version
- Only 2 gates (Update gate + Reset gate)
- Faster training, LSTM ku similar performance

---

## 5️⃣ GRU vs LSTM - Detailed Comparison

| **Aspect**      | **LSTM**                             | **GRU**                                   |
| --------------- | ------------------------------------ | ----------------------------------------- |
| **Gates**       | 3 Gates                              | 2 Gates                                   |
| **Cell State**  | Separate cell state irukum           | No separate cell state                    |
| **Parameters**  | More parameters                      | Less parameters                           |
| **Training**    | Slower                               | Faster                                    |
| **Memory**      | More memory needed                   | Less memory                               |
| **Performance** | Slightly better for complex tasks    | Similar, sometimes better                 |
| **Best For**    | Long sequences with complex patterns | Simpler sequences, faster training needed |

**When to use what?**

- **LSTM:** Long sequences, complex patterns, more data available
- **GRU:** Faster training needed, less data, simpler patterns

---

## 6️⃣ LSTM Applications - Real World Examples

### 📝 Natural Language Processing (NLP)

- **Language Translation:** English → Tamil conversion
- **Text Generation:** Next word/sentence prediction
- **Sentiment Analysis:** Movie reviews ah positive/negative nu classify
- **Chatbots:** Conversational AI

### 🎤 Speech Recognition

- **Voice Assistants:** Siri, Alexa, Google Assistant
- **Speech-to-Text:** Audio ah text ah convert pannum
- **Voice Commands:** "Hey Google, turn on lights"

### 🖼️ Image Processing

- **Image Captioning:** Image paathu description generate pannum
  - Example: "A dog playing in the park"
- **Video Analysis:** Frame-by-frame sequence analysis

### 🎵 Music Generation

- Notes sequence based ah new music create pannum
- Classical music patterns learn panni generate pannum

### 📈 Time Series Prediction

- **Stock Market:** Price predictions
- **Weather Forecast:** Temperature, rainfall predictions
- **Sales Forecasting:** Future sales predict pannum

### ✍️ Handwriting Recognition

- Handwritten text ah digital text ah convert pannum
- Signature verification

---

## 7️⃣ LSTM Advantages ✅

🎯 **Long-term Memory** - Vanishing gradient problem ah solve pannum  
🎯 **Selective Memory** - Gates use panni important info mattum remember pannum  
🎯 **Flexible** - Variable length sequences handle panlam  
🎯 **Versatile** - Text, audio, video, time-series data ellathukum use panlam  
🎯 **Better Context** - Past information ah effectively use pannum

---

## 8️⃣ LSTM Limitations ❌

❌ **Training Time** - RNN vida slow ah train aagum  
❌ **Computational Cost** - More parameters → More memory + processing power venum  
❌ **Still Gradient Issues** - Completely vanishing gradient solve aagala  
❌ **Overfitting** - Small datasets la overfit aaga chances irukum  
❌ **Sequential Processing** - Parallel processing pana mudiyathu (Transformers la idhukku solution)  
❌ **Hardware Demanding** - High memory bandwidth venum

---

## 9️⃣ Key Formulas - Quick Reference

### All Gates Together:

```
1. Forget Gate:     f_t = σ(W_f · [h_{t-1}, x_t] + b_f)
2. Input Gate:      i_t = σ(W_i · [h_{t-1}, x_t] + b_i)
3. Candidate:       C̃_t = tanh(W_C · [h_{t-1}, x_t] + b_C)
4. Cell State:      C_t = f_t · C_{t-1} + i_t · C̃_t
5. Output Gate:     o_t = σ(W_o · [h_{t-1}, x_t] + b_o)
6. Hidden State:    h_t = o_t · tanh(C_t)
```

### Activation Functions:

- **Sigmoid (σ):** 0 to 1 → Gate control (0=close, 1=open)
- **Tanh:** -1 to +1 → Value creation/transformation

---

## 🔟 Key Takeaways - Summary Points

🎯 **LSTM = RNN + Gates** (Long-term memory solution)  
🎯 **3 Gates:** Forget, Input, Output  
🎯 **2 States:** Cell State (long-term) + Hidden State (short-term)  
🎯 **Solves:** Vanishing gradient problem  
🎯 **Trade-off:** Better performance but slower training  
🎯 **Alternatives:** GRU (faster), Transformers (modern)  
🎯 **Best for:** Sequential data with long-term dependencies

---

# 💻 PyTorch Implementation Examples

## Example 1: Basic LSTM Layer

```python
import torch
import torch.nn as nn

# LSTM Parameters
input_size = 10      # Input features (e.g., word embedding dimension)
hidden_size = 20     # LSTM cell oda memory capacity
num_layers = 2       # Stacked LSTM layers
batch_size = 5       # Number of sequences
seq_length = 7       # Sequence length (e.g., 7 words)

# Create LSTM Layer
lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

# Random Input Data (Batch, Sequence, Features)
inputs = torch.randn(batch_size, seq_length, input_size)

# Initial States (Hidden State + Cell State)
h0 = torch.zeros(num_layers, batch_size, hidden_size)
c0 = torch.zeros(num_layers, batch_size, hidden_size)

# Forward Pass
output, (hn, cn) = lstm(inputs, (h0, c0))

print("Input Shape:", inputs.shape)          # [5, 7, 10]
print("Output Shape:", output.shape)         # [5, 7, 20]
print("Final Hidden State:", hn.shape)       # [2, 5, 20]
print("Final Cell State:", cn.shape)         # [2, 5, 20]

"""
Explanation:
- output: Each time step oda hidden state (ovvoru word kum)
- hn: Last time step oda hidden state mattum
- cn: Last time step oda cell state (long-term memory)
"""
```

---

## Example 2: Sentiment Analysis Model (Complete)

```python
import torch
import torch.nn as nn
import torch.optim as optim

class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout=0.5):
        super(SentimentLSTM, self).__init__()

        # Embedding Layer (Words ah vectors ah convert pannum)
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # LSTM Layer
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           batch_first=True,
                           dropout=dropout if n_layers > 1 else 0)

        # Fully Connected Layer (Classification)
        self.fc = nn.Linear(hidden_dim, output_dim)

        # Dropout for regularization
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        # text shape: [batch_size, seq_length]

        # Embedding
        embedded = self.dropout(self.embedding(text))
        # embedded shape: [batch_size, seq_length, embedding_dim]

        # LSTM
        output, (hidden, cell) = self.lstm(embedded)
        # output shape: [batch_size, seq_length, hidden_dim]
        # hidden shape: [n_layers, batch_size, hidden_dim]

        # Last layer oda last hidden state ah edukurom
        hidden = self.dropout(hidden[-1])  # [batch_size, hidden_dim]

        # Classification
        out = self.fc(hidden)  # [batch_size, output_dim]

        return out

# Model Parameters
VOCAB_SIZE = 10000      # Total unique words
EMBEDDING_DIM = 100     # Word vector size
HIDDEN_DIM = 256        # LSTM memory size
OUTPUT_DIM = 2          # Binary classification (Positive/Negative)
N_LAYERS = 2            # Stacked LSTM
DROPOUT = 0.5

# Create Model
model = SentimentLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, DROPOUT)

print(model)
print(f"\nTotal Parameters: {sum(p.numel() for p in model.parameters()):,}")

"""
Model Architecture:
1. Embedding: Words → Vectors
2. LSTM: Sequential processing
3. Dropout: Prevent overfitting
4. FC: Final classification

Example Usage:
- Input: "This movie is amazing!" → [45, 234, 12, 789, 23]
- Embedding: Each word ID → 100-dim vector
- LSTM: Process sequence → 256-dim hidden state
- FC: 256 → 2 (Positive/Negative scores)
- Softmax: Probabilities
"""
```

---

## Example 3: Text Generation (Character-Level LSTM)

```python
import torch
import torch.nn as nn
import numpy as np

class CharLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers):
        super(CharLSTM, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Character Embedding
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # LSTM
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, batch_first=True)

        # Output Layer
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden):
        # x shape: [batch_size, seq_length]

        # Embedding
        embed = self.embedding(x)
        # embed shape: [batch_size, seq_length, embedding_dim]

        # LSTM
        out, hidden = self.lstm(embed, hidden)
        # out shape: [batch_size, seq_length, hidden_dim]

        # Reshape for FC layer
        out = out.reshape(-1, self.hidden_dim)

        # FC
        out = self.fc(out)
        # out shape: [batch_size * seq_length, vocab_size]

        return out, hidden

    def init_hidden(self, batch_size, device):
        # Initialize hidden and cell states
        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        return (h0, c0)

# Example: Text Generation Function
def generate_text(model, start_text, char_to_idx, idx_to_char, length=100, temperature=1.0):
    model.eval()
    device = next(model.parameters()).device

    # Convert start text to indices
    chars = [char_to_idx[c] for c in start_text]
    input_seq = torch.tensor(chars).unsqueeze(0).to(device)  # [1, len(start_text)]

    # Initialize hidden state
    hidden = model.init_hidden(1, device)

    generated = start_text

    with torch.no_grad():
        for _ in range(length):
            # Forward pass
            out, hidden = model(input_seq, hidden)

            # Get last prediction
            logits = out[-1] / temperature
            probs = torch.softmax(logits, dim=0)

            # Sample next character
            next_idx = torch.multinomial(probs, 1).item()
            next_char = idx_to_char[next_idx]

            generated += next_char

            # Update input
            input_seq = torch.tensor([[next_idx]]).to(device)

    return generated

"""
Example Usage:
- Input: "Hello"
- Model predicts next character based on previous characters
- Output: "Hello world, this is an LSTM generated text..."

Temperature parameter:
- Low (0.5): More predictable, conservative
- High (1.5): More random, creative
"""
```

---

## Example 4: Bidirectional LSTM

```python
import torch
import torch.nn as nn

class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(BiLSTM, self).__init__()

        # Bidirectional LSTM
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers,
            batch_first=True,
            bidirectional=True  # Key parameter!
        )

        # FC layer (hidden_size * 2 because bidirectional)
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        # x shape: [batch_size, seq_length, input_size]

        # LSTM
        lstm_out, (hidden, cell) = self.lstm(x)
        # lstm_out shape: [batch_size, seq_length, hidden_size * 2]

        # Use last time step's output
        # lstm_out[:, -1, :] takes the last time step for all batches
        out = self.fc(lstm_out[:, -1, :])

        return out

# Create Bidirectional LSTM
bi_lstm = BiLSTM(input_size=50, hidden_size=128, output_size=3, num_layers=2)

# Test
test_input = torch.randn(32, 10, 50)  # [batch=32, seq=10, features=50]
output = bi_lstm(test_input)
print("Output shape:", output.shape)  # [32, 3]

"""
Bidirectional LSTM Advantages:
- Forward direction: Past context
- Backward direction: Future context
- Both combine → Better understanding

Example:
Sentence: "The cat sat on the mat"
- Forward LSTM: "The" → "cat" → "sat" (past context)
- Backward LSTM: "mat" → "the" → "on" (future context)
- Combined: Full sentence context for each word
"""
```

---

## Example 5: Many-to-Many LSTM (Sequence to Sequence)

```python
import torch
import torch.nn as nn

class Seq2SeqLSTM(nn.Module):
    def __init__(self, input_vocab_size, output_vocab_size, embedding_dim, hidden_dim):
        super(Seq2SeqLSTM, self).__init__()

        # Encoder
        self.encoder_embedding = nn.Embedding(input_vocab_size, embedding_dim)
        self.encoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Decoder
        self.decoder_embedding = nn.Embedding(output_vocab_size, embedding_dim)
        self.decoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Output
        self.fc = nn.Linear(hidden_dim, output_vocab_size)

    def forward(self, src, tgt):
        # src: Source sequence (e.g., English)
        # tgt: Target sequence (e.g., Tamil)

        # Encode
        src_embed = self.encoder_embedding(src)
        _, (hidden, cell) = self.encoder(src_embed)

        # Decode
        tgt_embed = self.decoder_embedding(tgt)
        decoder_out, _ = self.decoder(tgt_embed, (hidden, cell))

        # Output
        output = self.fc(decoder_out)

        return output

"""
Seq2Seq Application: Language Translation
Input: "I love programming" (English)
Process:
1. Encoder: English sentence ah encode pannum → context vector
2. Decoder: Context vector use panni Tamil la decode pannum
Output: "Naan programming ah love pannuren" (Tamil)
"""
```

---

## 🎓 Practice Exercise

```python
# Exercise: Create a simple LSTM for sequence classification

# TODO 1: Define LSTM model with following specs:
#   - Vocab size: 5000
#   - Embedding dimension: 100
#   - Hidden dimension: 128
#   - Output classes: 3 (Multi-class classification)
#   - Layers: 2

# TODO 2: Create sample input data
#   - Batch size: 16
#   - Sequence length: 20

# TODO 3: Forward pass and print shapes

# TODO 4: Calculate total parameters in model

# Solution below (try first before looking!)
```

<details>
<summary>Solution (Click to expand)</summary>

```python
import torch
import torch.nn as nn

# Solution
class SimpleLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, n_layers):
        super(SimpleLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embed = self.embedding(x)
        _, (hidden, _) = self.lstm(embed)
        out = self.fc(hidden[-1])
        return out

# Create model
model = SimpleLSTM(5000, 100, 128, 3, 2)

# Sample input
batch_size, seq_len = 16, 20
x = torch.randint(0, 5000, (batch_size, seq_len))

# Forward pass
output = model(x)
print("Output shape:", output.shape)  # [16, 3]

# Total parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")



